# 37009 Workshop Week 2: Review of Financial Market Instruments (Part 1)

**Note:** Variable names will be reused across the entire workshop exercise, so make sure that you are running the correct code section for each exercise.

## 1 Bonds 

We first construct a table of remaining cash flows indexed by the time (in years) from the valuation date in which these cash flows are realized. Since the bond matures on 30 Sep 2025 and payments are semi-annual, the remaining payments as of 15 Aug 2023 are on 30 Mar 2024, 30 Sep 2024, 30 Mar 2025, and 30 Sep 2025, with the principal due also on 30 Sep 2025.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import math
from datetime import datetime
from datetime import timedelta

# Bond parameters
coupon_rate = 0.05
coupon_freq = 2
principal = 100
date_today = datetime(2023, 8, 15)

# Remaining payment dates
payment_dates = np.array([datetime(2023, 9, 30), datetime(2024, 3, 30), datetime(2024, 9, 30), datetime(2025, 3, 30),
                         datetime(2025, 9, 30), datetime(2025, 9, 30)])
payment_dates_from_today = payment_dates - date_today

In [3]:
# Construct a table of cash flows index by the time from today (in years)
cash_flows = np.append(np.array([principal * coupon_rate / coupon_freq] * 5), [principal])
bond_summary = pd.DataFrame({'Time': payment_dates_from_today, 'CF': cash_flows})
bond_summary['Time'] = bond_summary['Time'].dt.days / 360
print(bond_summary)

       Time     CF
0  0.127778    2.5
1  0.633333    2.5
2  1.144444    2.5
3  1.647222    2.5
4  2.158333    2.5
5  2.158333  100.0


Next, we encode the zero rate data.

In [4]:
# Zero rates
zero_rates = pd.DataFrame({'Tenor': [0, 1/12, 3/12, 6/12, 1, 2, 3, 4],
                          'Rate': [3.31, 4.05, 4.12, 4.58, 4.95, 5.44, 5.57, 5.82]})
zero_rates['Rate'] = zero_rates['Rate'] / 100
print(zero_rates)

      Tenor    Rate
0  0.000000  0.0331
1  0.083333  0.0405
2  0.250000  0.0412
3  0.500000  0.0458
4  1.000000  0.0495
5  2.000000  0.0544
6  3.000000  0.0557
7  4.000000  0.0582


Observe that *none* of the bond payment dates coincide with the tenors of the zero rates. We thus have to interpolate from our zero curve the zero rates that are appropriate for discounting the cash flows from the bond.

In [5]:
# Interpolate appropriate zero rates for discounting CFs
interp_rates = np.interp(x = bond_summary['Time'], xp = zero_rates['Tenor'], fp = zero_rates['Rate'])
bond_summary['Rate'] = interp_rates
print(bond_summary)

       Time     CF      Rate
0  0.127778    2.5  0.040687
1  0.633333    2.5  0.046787
2  1.144444    2.5  0.050208
3  1.647222    2.5  0.052671
4  2.158333    2.5  0.054606
5  2.158333  100.0  0.054606


We are now ready to price the bond. The next steps are to calculate the discounted cash flows and to find the sum of the discounted cash flows.

In [6]:
# Compute the discount factor (continuous compounding)
bond_summary['DF'] = np.exp(-bond_summary['Rate'] * bond_summary['Time'])

# Compute discounted cash flows
bond_summary['DCF'] = bond_summary['CF'] * bond_summary['DF']

# Compute bond price
bond_price = np.sum(bond_summary['DCF'])
print("Bond price is AUD", bond_price)

print("Below is a summary of the cash flows in the bond")
print(bond_summary)

Bond price is AUD 100.67100161819766
Below is a summary of the cash flows in the bond
       Time     CF      Rate        DF        DCF
0  0.127778    2.5  0.040687  0.994815   2.487037
1  0.633333    2.5  0.046787  0.970803   2.427008
2  1.144444    2.5  0.050208  0.944160   2.360399
3  1.647222    2.5  0.052671  0.916896   2.292239
4  2.158333    2.5  0.054606  0.888823   2.222057
5  2.158333  100.0  0.054606  0.888823  88.882262


If the zero rates are semi-annually compounded, the calculation of the discount factors will change. This will be left as an exercise.

**Keep in mind for future discussions:** The bond price can be expressed as a linear combination of zero-coupon bond prices, 

$$p(0) = 2.5 P(0,0.128) + 2.5 P(0,0.633) + 2.5 P(0, 1.14) + 2.5 P(0,1.65) + 102.5 P(2.16).$$

## 2 Forward Rate Agreements

We consider a $(5 \times 14)$ FRA that is set today, 15 August 2023, on a notional of $L = 1.25$ million AUD.

### Computing the FRA fixed/contractual rate

The fixed or contractual rate for the FRA is simply the simple forward rate for the period $[5/12, 14/12]$ implied by the simple zero rates observed today.

In [10]:
# FRA parameters
T1 = 5/12
T2 = 14/12
notional = 1250000

# Encode zero rates today
zero_rates = pd.DataFrame({'Tenor': [1/12, 3/12, 6/12, 1, 1.5],
                          'Rate': [1.75, 2.25, 3.70, 4.35, 5.15]})
zero_rates['Rate'] = zero_rates['Rate'] / 100

Note that the 5-month and 14-month zero rates are not available, so we interpolate these from the given zero rates.

In [11]:
# Interpolate required simple zero rates
tenor_req = np.array([T1, T2])
interp_rates = np.interp(x = tenor_req, xp = zero_rates['Tenor'], fp = zero_rates['Rate'])

# Create summary of FRA characteristics
fra_summary = pd.DataFrame({'Time': [T1, T2], 'Rate': interp_rates})
fra_summary['DF'] = 1 / (1 + fra_summary['Rate'] * fra_summary['Time'])
print(fra_summary)

       Time      Rate        DF
0  0.416667  0.032167  0.986774
1  1.166667  0.046167  0.948892


We can now compute the foward rate for the period $[5/12, 14/12]$ using the interpolated zero rates using the formula

$$R_{\text{fixed}} = R_F(0,5/12,14/12) = \left(\frac{P(0,5/12)}{P(0,14/12)} - 1\right) \frac{1}{14/12 - 5/12}$$

In [12]:
# Compute contractual forward rate
fwd_rate_contract = (fra_summary.iloc[0,2] / fra_summary.iloc[1,2] - 1) / (T2 - T1)
fwd_rate_contract

0.05323100116494187

### Valuation of the FRA

Two months after the issuance of the FRA (15 October 2023), we compute the value of the contract. It is sufficient to compute the value of the FRA to the fixed rate payer since the value to the floating rate payer is the additive inverse.

As of the valuation date, the FRA has 3 months left in its life ($T_1 = 3/12$) and matures 12 months from now ($T_2 = 1$). Fortunately, the required information is all in the given term structure of zero rates, so no interpolation is required. We only need to compute the forward rate for the period $[3/12, 1]$ based on zero rates at the valuation date.

In [13]:
# Create summary of FRA characteristics at valuation date (manually entered)
T1_val = 3/12
T2_val = 1
fra_summary2 = pd.DataFrame({'Time': [T1_val, T2_val], 'Rate': [2.15, 4.10]})
fra_summary2['Rate'] = fra_summary2['Rate'] / 100
fra_summary2['DF'] = 1 / (1 + fra_summary2['Rate'] * fra_summary2['Time'])
print(fra_summary2)

   Time    Rate        DF
0  0.25  0.0215  0.994654
1  1.00  0.0410  0.960615


In [14]:
# Compute the forward rate observed today
fwd_rate = (fra_summary2.iloc[0,2] / fra_summary2.iloc[1,2] - 1) / (T2_val - T1_val)
fwd_rate

0.04724605246798467

Using the formula

$$p(t) = L(R_F - R_{\text{fixed}})(T_2 - T_1) P(t,T_2),$$

the value of the FRA today to the fixed rate payer is:

In [15]:
fra_value = notional * (fwd_rate - fwd_rate_contract) * (T2_val - T1_val) * fra_summary2.iloc[1,2]
fra_value

-5389.903365415351

The FRA can also be valued using the alternative formula 

$$p(t) = LP(t,T_1) - L(1 + R_{\text{fixed}}(T_2 - T-1)) P(t,T_2).$$

We verify that we obtain the same result (with some rounding issues).

In [16]:
fra_value2 = notional * fra_summary2.iloc[0,2] - notional * (1 + fwd_rate_contract * (T2_val - T1_val)) * fra_summary2.iloc[1,2]
fra_value2

-5389.903365415521

### Cash Settlement at Expiry of the FRA

This calculation will be left as an exercise, as it is a straightforward application of the formula 

$$L(R_{\text{float}} - R_{\text{fixed}})(T_2 - T_1),$$

where $T_2 - T_1 = 9/12$ years. At the expiry of the FRA (15 January 2024, 5 months from 15 August 2023), it is known that the 6-month zero rate is 3.65% and the 1-year zero rate is 4.05%. In the formula above, $R_{\text{float}}$ is taken to be the realized 9-month rate at expirty, which must be interpolated from the given data. With $R_{\text{fixed}}$ taken to be **fwd_rate_contract** above.

## 3 Interest Rate Swaps

We first identify the IRS contract features and compute the required forward rates.

In [17]:
# IRS contract features
date_today = datetime(2017, 6, 8)
n_pmt_dates = 3
resetpmt_dates = np.array([datetime(2017, 7, 15), datetime(2018, 1, 15), datetime(2018, 7, 15), datetime(2019, 1, 15)])
resetpmt_dates_from_today = resetpmt_dates - date_today

# IRS cash flow summary
irs_summary = pd.DataFrame({'Time': resetpmt_dates_from_today})
irs_summary['Time'] = irs_summary['Time'].dt.days / 360
irs_summary['Event'] = np.array(['Reset', 'Payment', 'Payment', 'Payment'])

# Calculate time differences between successive reset and payment dates
irs_summary['TimeDiff'] = np.array([0] * (n_pmt_dates + 1))

for i in range(1, n_pmt_dates+1):
    irs_summary.loc[irs_summary.index[i],'TimeDiff'] = irs_summary.loc[irs_summary.index[i],'Time'] - irs_summary.loc[irs_summary.index[i-1],'Time']
    
print(irs_summary)
    

       Time    Event  TimeDiff
0  0.102778    Reset  0.000000
1  0.613889  Payment  0.511111
2  1.116667  Payment  0.502778
3  1.627778  Payment  0.511111


Next, we need to identify the applicable zero rates observed today with tenor corresponding to the reset or payment dates. Interpolation is necessary.

In [18]:
# Simple zero rates today
tenor = np.array([1/12, 3/12, 6/12, 1, 1.5, 2])
rates = np.array([1.85, 2.11, 2.65, 3.17, 3.55, 4.05]) / 100
zero_rates = pd.DataFrame({'Tenor': tenor, 'Rate': rates})
print(zero_rates)

      Tenor    Rate
0  0.083333  0.0185
1  0.250000  0.0211
2  0.500000  0.0265
3  1.000000  0.0317
4  1.500000  0.0355
5  2.000000  0.0405


In [19]:
# Interpolate zero rates for IRS reset and payment dates
interp_rates = np.interp(x = irs_summary['Time'], xp = zero_rates['Tenor'], fp = zero_rates['Rate'])
irs_summary['Rate'] = interp_rates

# Compute discount factors
irs_summary['DF'] = 1 / (1 + irs_summary['Rate'] * irs_summary['Time'])

print(irs_summary)

       Time    Event  TimeDiff      Rate        DF
0  0.102778    Reset  0.000000  0.018803  0.998071
1  0.613889  Payment  0.511111  0.027684  0.983289
2  1.116667  Payment  0.502778  0.032587  0.964889
3  1.627778  Payment  0.511111  0.036778  0.943515


Finally, we compute the forward rates observed today for all periods in between the reset and payment dates.

In [20]:
# Compute forward rates
# n_pmt_dates = irs_summary.shape[0] - 1 # Number of rows minus 1
# print(n_pmt_dates)

# # Some testing
# # Vectorized calculation of forward rates (alternatively, for-loops can be used)
# print(irs_summary.iloc[1:n_pmt_dates+1, 3]) # Gives P(t,T_{i+1}) for i=0,1,2
# print(irs_summary.iloc[0:n_pmt_dates, 3]) # Gives P(t,T_i)) for i=0,1,2
# print(np.array(irs_summary.iloc[1:n_pmt_dates+1, 0]) - np.array(irs_summary.iloc[0:n_pmt_dates, 0])) # Gives T_{i+1} - T_i for i=0,1,2
# fwd_rates = (np.array(irs_summary.iloc[0:n_pmt_dates, 3]) / np.array(irs_summary.iloc[1:n_pmt_dates+1, 3]) - 1) \
# / (np.array(irs_summary.iloc[1:n_pmt_dates+1, 0]) - np.array(irs_summary.iloc[0:n_pmt_dates, 0])) 

fwd_rate_num = np.array(irs_summary.iloc[0:n_pmt_dates, 4]) / np.array(irs_summary.iloc[1:n_pmt_dates+1, 4]) - 1
fwd_rate_denom = np.array(irs_summary.iloc[1:n_pmt_dates+1, 2])
fwd_rate = fwd_rate_num / fwd_rate_denom
fwd_rate = np.append([0], fwd_rate)

# fwd_rate_num = np.array(irs_summary.truncate(before = 0, after = n_pmt_dates-1))

irs_summary['ForwardRate'] = fwd_rate
print(irs_summary)

       Time    Event  TimeDiff      Rate        DF  ForwardRate
0  0.102778    Reset  0.000000  0.018803  0.998071     0.000000
1  0.613889  Payment  0.511111  0.027684  0.983289     0.029413
2  1.116667  Payment  0.502778  0.032587  0.964889     0.037928
3  1.627778  Payment  0.511111  0.036778  0.943515     0.044322


### Calculating the Swap Rate

Using the zero rates observed today (8 June 2017), we need compute the swap rate using the formula

$$s(t) = \frac{\sum_{i=0}^{N-1} R_F(t;T_i,T_{i+1})(T_{i+1} - T_i) P(t, T_{i+1})}{\sum_{i=0}^{N-1}(T_{i+1} - T_i)P(t,T_{i+1})}.$$

In [21]:
# Calculate floating leg summands
irs_summary['FloatComp'] = irs_summary['ForwardRate'] * irs_summary['TimeDiff'] * irs_summary['DF']

# Calculate fixed leg summands
irs_summary['FixedComp'] = irs_summary['TimeDiff'] * irs_summary['DF']

# Calculate swap rate
swap_rate = sum(irs_summary['FloatComp']) / sum(irs_summary['FixedComp'])
print('The swap rate is:', swap_rate * 100, '%')

The swap rate is: 3.711434280134335 %


### Valuing the IRS



In [22]:
# Contract features as of today = 15 July 2017
date_today = datetime(2017, 7, 15)
n_pmt_dates = 3
resetpmt_dates = np.array([datetime(2017, 7, 15), datetime(2018, 1, 15), datetime(2018, 7, 15), datetime(2019, 1, 15)])
resetpmt_dates_from_today = resetpmt_dates - date_today

# IRS cash flow summary
irs_summary = pd.DataFrame({'Time': resetpmt_dates_from_today})
irs_summary['Time'] = irs_summary['Time'].dt.days / 360
irs_summary['Event'] = np.array(['Reset', 'Payment', 'Payment', 'Payment'])

# Calculate time differences between successive reset and payment dates
irs_summary['TimeDiff'] = np.array([0] * (n_pmt_dates + 1))

for i in range(1, n_pmt_dates+1):
    irs_summary.loc[irs_summary.index[i],'TimeDiff'] = irs_summary.loc[irs_summary.index[i],'Time'] - irs_summary.loc[irs_summary.index[i-1],'Time']
    
print(irs_summary)

       Time    Event  TimeDiff
0  0.000000    Reset  0.000000
1  0.511111  Payment  0.511111
2  1.013889  Payment  0.502778
3  1.525000  Payment  0.511111


In [23]:
# Simple zero rates today
tenor = np.array([1/12, 3/12, 6/12, 1, 1.5, 2])
rates = np.array([2.10, 2.36, 2.90, 3.42, 3.80, 4.30]) / 100
zero_rates = pd.DataFrame({'Tenor': tenor, 'Rate': rates})
print(zero_rates)

      Tenor    Rate
0  0.083333  0.0210
1  0.250000  0.0236
2  0.500000  0.0290
3  1.000000  0.0342
4  1.500000  0.0380
5  2.000000  0.0430


**Note:** Due to the day-count convention, the payment dates do not coincide with the tenor of the zero rates observed today. However, in terms of months, the payment dates match the tenors. In this case, we will assume that the zero rates for the 6-month, 1-year, and 1.5-year tenors apply to our payment dates (i.e. we do not need to interpolate).

In [24]:
# Append zero rates and corresponding discount factors to the irs_summary table
irs_summary['Rate'] = np.array([0, 2.90, 3.42, 3.80]) / 100
irs_summary['DF'] = 1 / (1 + irs_summary['Rate'] * irs_summary['Time'])
print(irs_summary)

       Time    Event  TimeDiff    Rate        DF
0  0.000000    Reset  0.000000  0.0000  1.000000
1  0.511111  Payment  0.511111  0.0290  0.985394
2  1.013889  Payment  0.502778  0.0342  0.966487
3  1.525000  Payment  0.511111  0.0380  0.945224


In [25]:
# Compute forward rates
fwd_rate_num = np.array(irs_summary.iloc[0:n_pmt_dates, 4]) / np.array(irs_summary.iloc[1:n_pmt_dates+1, 4]) - 1
fwd_rate_denom = np.array(irs_summary.iloc[1:n_pmt_dates+1, 2])
fwd_rate = fwd_rate_num / fwd_rate_denom
fwd_rate = np.append([0], fwd_rate)

# fwd_rate_num = np.array(irs_summary.truncate(before = 0, after = n_pmt_dates-1))

irs_summary['ForwardRate'] = fwd_rate
print(irs_summary)

       Time    Event  TimeDiff    Rate        DF  ForwardRate
0  0.000000    Reset  0.000000  0.0000  1.000000     0.000000
1  0.511111  Payment  0.511111  0.0290  0.985394     0.029000
2  1.013889  Payment  0.502778  0.0342  0.966487     0.038909
3  1.525000  Payment  0.511111  0.0380  0.945224     0.044012


We now compute the value of the floating and fixed legs by calculating the summands in the formulas below:
\begin{align*}
V_{\text{float}}(t) & = L\sum_{i=0}^{N-1} R_F(t;T_i,T_{i+1})(T_{i+1} - T_i) P(t, T_{i+1}) \\
V_{\text{fixed}}(t) & = LK\sum_{i=0}^{N-1}(T_{i+1} - T_i)P(t,T_{i+1})
\end{align*}

In [26]:
# Compute floating and fixed leg summands
irs_summary['FloatComp'] = irs_summary['ForwardRate'] * irs_summary['TimeDiff'] * irs_summary['DF']
irs_summary['FixedComp'] = irs_summary['TimeDiff'] * irs_summary['DF']

# Compute value of the IRS to the fixed rate payer
notional = 5000000
irs_value = notional * sum(irs_summary['FloatComp']) - notional * swap_rate * sum(irs_summary['FixedComp'])

print(irs_summary)
print('The value of the IRS to the fixed rate payer is AUD', irs_value)

       Time    Event  TimeDiff    Rate        DF  ForwardRate  FloatComp  \
0  0.000000    Reset  0.000000  0.0000  1.000000     0.000000   0.000000   
1  0.511111  Payment  0.511111  0.0290  0.985394     0.029000   0.014606   
2  1.013889  Payment  0.502778  0.0342  0.966487     0.038909   0.018907   
3  1.525000  Payment  0.511111  0.0380  0.945224     0.044012   0.021263   

   FixedComp  
0   0.000000  
1   0.503646  
2   0.485928  
3   0.483115  
The value of the IRS to the fixed rate payer is AUD 589.3439175679814


## 4 Currency Forwards

We first encode the assumed zero rates for the foreign (South Korean) and domestic (Australian) markets.

In [27]:
# Zero rates for domestic and foreign markets
zero_rates = pd.DataFrame({'Tenor':[3/12, 6/12, 9/12],
                          'Domestic':[0.0286, 0.0296, 0.0306],
                          'Foreign':[0.0425, 0.0464, 0.0489]})
print(zero_rates)

   Tenor  Domestic  Foreign
0   0.25    0.0286   0.0425
1   0.50    0.0296   0.0464
2   0.75    0.0306   0.0489


### Forward Exchange Rate

Today, 15 August 2023, we enter a forward contract with delivery date 1 March 2024. We first determine how many years the contract has between inception and delivery.

In [28]:
# Dates
date_today = datetime(2023, 8, 15)
delivery_date = datetime(2024, 3, 1)
time_diff = delivery_date - date_today
time_diff = time_diff.days / 360
print(time_diff)

0.5527777777777778


We thus have to interpolate the applicable interest rates from the given data.

In [29]:
# Interpolate rates
r_dom = np.interp(x = time_diff, xp = zero_rates['Tenor'], fp = zero_rates['Domestic'])
r_for = np.interp(x = time_diff, xp = zero_rates['Tenor'], fp = zero_rates['Foreign'])
print(r_dom, r_for)

0.02981111111111111 0.04692777777777778


The spot exchange rate is 865 KRW per AUD. Thus, $S(0) = \frac{1}{865}$ AUD per KRW. We can then compute the forward exchange rate as 

$$F(0,T) = e^{(r - r_f) T} S(0).$$

In [30]:
# Spot foreign exchange rate
spot_fx = 1 / 865

# Forward exchange rate
fwd_fx = np.exp((r_dom - r_for) * time_diff) * spot_fx

The computed forward exchange rate here will serve as the contractual rate for the currency forward.

In [31]:
# Contract forward exchange rate
contract_fwd_fx = fwd_fx

### Forward Contract Valuation

We now state the valuation date and the time until the delivery date. The term structure of interest rates in both markets are assumed to be the same as the previous item, but we need to determine the applicable rates. We also note the spot foreign exchange on the valuation date.

In [32]:
# Dates
date_today = datetime(2023, 10, 15)
time_diff = delivery_date - date_today
time_diff = time_diff.days / 360

# Interpolate rates
r_dom = np.interp(x = time_diff, xp = zero_rates['Tenor'], fp = zero_rates['Domestic'])
r_for = np.interp(x = time_diff, xp = zero_rates['Tenor'], fp = zero_rates['Foreign'])

# Spot foreign exchange rate
spot_fx = 1 / 880

# Forward exchange rate at valuation date
fwd_fx = np.exp((r_dom - r_for) * time_diff) * spot_fx

We are now ready to compute the value of the forward contract for the long position.

In [33]:
# Forward contract value
value = (fwd_fx - contract_fwd_fx) * np.exp(-r_dom * time_diff)
print('The value of the forward contract is AUD', value, 'per KRW or AUD', 25000000 * value)

The value of the forward contract is AUD -1.5355251240873522e-05 per KRW or AUD -383.88128102183805
